In [1]:
import csv
import random
import numpy as np # linear algebra
import pandas as pd 

from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
from joblib import load

input_path = ''
my_model = load('109550110_LR_model.joblib')

c:\Users\R5 3600\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
def data_extract(data_df):
    data = data_df.iloc[:, :].values
    for i, x in enumerate(data_df.dtypes):
        if(x == 'object'): # [product_code, attribute_0, attribute_1]
            if(len(data[0, i])) == 1: #product_code
                data[:, i] = data[:, i].astype('str').view(np.int32) - ord('A')
            else: #attribute
                temp = data[:, i].astype('str')
                data[:, i] = np.array([int(x[-1]) for x in temp])
                
        elif i != 0 and i != len(data[0]) - 1: # not in column 'ID' or 'failure'
            imr = SimpleImputer(strategy = 'median')
            data[:, i] = imr.fit_transform(data[:, i].reshape(-1, 1)).reshape(len(data[:, i]))
            #scaler = StandardScaler()
            #data[:, i] = scaler.fit_transform(data[:, i].reshape(-1, 1)).reshape(len(data[:, i]))
            
    return data[:, 1:] # get rid of [ID]

In [3]:
test_df = pd.read_csv(input_path +'test.csv')
test_df.insert(len(test_df.columns), 'dummy', 0)
ID = test_df.iloc[:,0].values

test = data_extract(test_df)
print(test.shape)

(20775, 25)


In [4]:
test_x = test[:, :-1].astype('float')

pred_prob = my_model.predict_proba(test_x)
prob = pd.DataFrame({'id':ID, 'failure': pred_prob[:, 1].astype(float)})
prob.to_csv('109550110_submission.csv', index=False)